In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import matthews_corrcoef, confusion_matrix

## Carga de bandas para las máscaras originales

In [ ]:
bandas = {'Bands_10':['B4 Red [665 nm]', 'B3 Green [560 nm]', 'B2 Blue [490 nm]', 'B8 Near infrared [842 nm]'],
          'Bands_20':['B8A Vegetation classification [865 nm]', 'B11 Snow ice cloud discrimination [1610 nm]', 
                         'B12 Snow ice cloud discrimination [2190 nm]']}

array_bandas = {}

indices = {}

def CargaBandas(archivo):
    
    for resolucion in bandas:
        
        netcdf = Dataset(os.path.join('/mnt/Oliver/Ponteareas', archivo, '{}.nc'.format(resolucion)), 'r')
        
        for banda in bandas[resolucion]:
               
            array = netcdf.variables[banda][:]

            array_bandas[banda.split(' ')[0]] = array

            
    return array_bandas


def CargaIndices(archivo):
    
    
    ndwi = (array_bandas['B3'] - array_bandas['B8'])/(array_bandas['B3'] + array_bandas['B8'])
    ndwi_res = cv2.resize(ndwi, (5490,5490), interpolation = cv2.INTER_CUBIC)
    indices['ndwi'] = ndwi_res
    ndwi_res[ndwi_res >= 0.2] = np.nan
    ndwi_res = np.ma.masked_where(condition=np.isnan(ndwi_res), a=ndwi_res)
      
    B11 = cv2.resize(array_bandas['B11'], (10980,10980), interpolation=cv2.INTER_CUBIC)
    ndsi = (array_bandas['B3'] - B11)/(array_bandas['B3'] + B11)
    indices['ndsi'] = ndsi
    ndsi = cv2.resize(ndsi, (5490,5490), interpolation = cv2.INTER_CUBIC)

    B4 = cv2.resize(array_bandas['B4'], (5490,5490), interpolation = cv2.INTER_CUBIC)
    mask_potential_cloud = (B4 > 0.07) * (B4 < 0.25) # are passed to Step 1.b
    mask_cloud = B4 > 0.25 # Clouds , are passed to step2
    mask_cloud = mask_cloud + (mask_potential_cloud * (ndsi > -0.16))
    
       
    nbr = (array_bandas['B8A'] - array_bandas['B12'])/(array_bandas['B8A'] + array_bandas['B12'])
    nbr[ndwi_res.mask] = 1
    nbr[mask_cloud] = 1
    indices['nbr'] = nbr
    
    ndvi = (array_bandas['B8'] - array_bandas['B4'])/(array_bandas['B8'] + array_bandas['B4'])
    ndvi_res = cv2.resize(ndvi, (5490,5490), interpolation = cv2.INTER_CUBIC)
    ndvi_res[ndwi_res.mask] = 1
    ndvi_res[mask_cloud] = 1
    indices['ndvi'] = ndvi_res
    
    
    return indices



 
archivo = 'S2A_MSIL2A_20171012T112111_N0205_R037_T29TNG_20171012T112713'

array_bandas = CargaBandas(archivo)

indices = CargaIndices(archivo)

In [ ]:
nbr_pre = indices['nbr']
ndvi_pre = indices['ndvi']

In [ ]:
nbr_mask = indices['nbr']
nbr_mask[nbr_mask >= -0.2] = 0
nbr_mask[nbr_mask <= -0.2] = 1

ndvi_mask = indices['ndvi']
ndvi_mask[ndvi_mask <= 0] = 0
ndvi_mask[ndvi_mask > 0] = 1


## Carga de bandas y cálculo de índices

In [ ]:
bandas = {'Bands_10':['B4 Red [665 nm]', 'B3 Green [560 nm]', 'B2 Blue [490 nm]', 'B8 Near infrared [842 nm]'],
          'Bands_20':['B8A Vegetation classification [865 nm]', 'B11 Snow ice cloud discrimination [1610 nm]', 
                         'B12 Snow ice cloud discrimination [2190 nm]']}

array_bandas = {}

indices = {}

def CargaBandas(archivo):
    
    for resolucion in bandas:
        
        netcdf = Dataset(os.path.join('/mnt/Oliver/Ponteareas', archivo, '{}.nc'.format(resolucion)), 'r')
        
        for banda in bandas[resolucion]:
               
            array = netcdf.variables[banda][:]

            array_bandas[banda.split(' ')[0]] = array

            
    return array_bandas


def CargaIndices(archivo):
    
    
    ndwi = (array_bandas['B3'] - array_bandas['B8'])/(array_bandas['B3'] + array_bandas['B8'])
    ndwi_res = cv2.resize(ndwi, (5490,5490), interpolation = cv2.INTER_CUBIC)
    indices['ndwi'] = ndwi_res
    ndwi_res[ndwi_res >= 0.2] = np.nan
    ndwi_res = np.ma.masked_where(condition=np.isnan(ndwi_res), a=ndwi_res)
      
    B11 = cv2.resize(array_bandas['B11'], (10980,10980), interpolation=cv2.INTER_CUBIC)
    ndsi = (array_bandas['B3'] - B11)/(array_bandas['B3'] + B11)
    indices['ndsi'] = ndsi
    ndsi = cv2.resize(ndsi, (5490,5490), interpolation = cv2.INTER_CUBIC)

    B4 = cv2.resize(array_bandas['B4'], (5490,5490), interpolation = cv2.INTER_CUBIC)
    mask_potential_cloud = (B4 > 0.07) * (B4 < 0.25) # are passed to Step 1.b
    mask_cloud = B4 > 0.25 # Clouds , are passed to step2
    mask_cloud = mask_cloud + (mask_potential_cloud * (ndsi > -0.16))
    
       
    nbr = (array_bandas['B8A'] - array_bandas['B12'])/(array_bandas['B8A'] + array_bandas['B12'])
    nbr[ndwi_res.mask] = 1
    nbr[mask_cloud] = 1
    indices['nbr'] = nbr
    
    ndvi = (array_bandas['B8'] - array_bandas['B4'])/(array_bandas['B8'] + array_bandas['B4'])
    ndvi_res = cv2.resize(ndvi, (5490,5490), interpolation = cv2.INTER_CUBIC)
    ndvi_res[ndwi_res.mask] = 1
    ndvi_res[mask_cloud] = 1
    indices['ndvi'] = ndvi_res
    
    
    return indices



 
archivo = 'S2A_MSIL2A_20171022T112121_N0205_R037_T29TNG_20171022T112802'

array_bandas = CargaBandas(archivo)

indices = CargaIndices(archivo)

## Resta de Imágenes

In [ ]:
nbr_post = indices['nbr']

nbr = nbr_post - nbr_pre

nbr[nbr >= -0.2] = 0
nbr[nbr <= -0.2] = 1

plt.imshow(nbr, origin='lower', cmap='gray')

## Método no supervisado, K-Means

In [ ]:
img =  np.zeros((indices['nbr'].shape[0], indices['nbr'].shape[1], 2))
img[:,:,0] = indices['nbr']
img[:,:,1] = indices['ndvi']

new_shape = (img.shape[0] * img.shape[1], img.shape[2])

X = img[:, :, :2].reshape(new_shape)
    
k_means = KMeans(n_clusters=5).fit(X)
X_cluster = k_means.labels_
X_cluster = X_cluster.reshape(img[:, :, 0].shape)
    
plt.figure(figsize=(10,10))
plt.imshow(X_cluster, origin='lower')
plt.colorbar(ticks=np.linspace(0,5,5))

## Método supervisado, Regresión Logística

In [ ]:
def SupervisedClustering(array_bandas, indice, mascara):
    
    img =  np.zeros((array_bandas['B8A'].shape[0], array_bandas['B8A'].shape[1], 2))
    img[:,:,0] = array_bandas['B8A']
    img[:,:,1] = array_bandas['B12']
    
    new_shape = (img.shape[0] * img.shape[1], img.shape[2])
    X = img[:, :, :2].reshape(new_shape)
    y_f = mascara.flatten()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_f, test_size=0.8, random_state=42)
    
    model=LogisticRegression(random_state=42).fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    print('Accuracy of test: {}'.format(accuracy_score(y_test, y_pred)))
    
    mask_pred = model.predict(X)
    print('Accuracy of complete mask: {}'.format(accuracy_score(y_f, mask_pred)))
    mask_pred = mask_pred.reshape(indice.shape)
      
    fig, axs = plt.subplots(1, 2, figsize=(16, 8))
    axs = axs.flatten()

    axs[0].imshow(mascara, origin='lower left', cmap='gray')
    axs[0].set_title('original Mask', fontsize=16)

    axs[1].imshow(mask_pred, origin='lower left', cmap='gray')
    axs[1].set_title('pred Mask', fontsize=16)
    

SupervisedClustering(array_bandas, indices['nbr'], nbr_mask)